In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
%run ~/violin-renderer/src/misc/parse.ipynb
# %run ~/violin-renderer/src/misc/randomizer.ipynb

In [ ]:
HOME_DIR = os.path.expanduser("~")

In [ ]:
# Load all the datasets
training_X, training_y, testing_X, testing_y = load_from_paths(HOME_DIR)

# Note: for training we just need to combine every song into an array
# We remake so we have 2 different models for (onset, pitch) -> start and (offset, pitch) -> end
training_X_onset = [(pair[0], pair[2]) for song in training_X for pair in song]
training_X_offset = [(pair[1], pair[2]) for song in training_X for pair in song]
training_y_start = [pair[0] for song in training_y for pair in song]
training_y_end = [pair[1] for song in training_y for pair in song]

# Remake based on the same logic as above, but we dont combine every songs
testing_X_onset = [[(pair[0], pair[2]) for pair in song] for song in testing_X]
testing_X_offset = [[(pair[1], pair[2]) for pair in song] for song in testing_X]
testing_y_start = [[pair[0] for pair in song] for song in testing_y]
testing_y_end = [[pair[1] for pair in song] for song in testing_y]

In [ ]:
# Linear regression model to predict onset
def LR_onset():
    model_onset = linear_model.LinearRegression()
    model_onset.fit(training_X_onset, training_y_start)
    return model_onset

# Linear regression model to predict offset
def LR_offset():
    model_offset = linear_model.LinearRegression()
    model_offset.fit(training_X_offset, training_y_end)
    return model_offset

In [ ]:
def LR_MSE(model_onset, model_offset):
    errors = []
    total = len(testing_X_onset)
    for i in range(total):
        Y_onset_pred = model_onset.predict(testing_X_onset[i])
        Y_offset_pred = model_offset.predict(testing_X_offset[i])

        onset_error = mean_squared_error(testing_y_start[i], Y_onset_pred)
        offset_error = mean_squared_error(testing_y_end[i], Y_offset_pred)

        errors.append((onset_error + offset_error) / 2)
    
    return errors